In [1]:
import pandas as pd
import numpy as np
import time
import functools

In [2]:
df = pd.read_csv("dataset/walk.csv")

In [3]:
df.head()

,id,frame,r_hip_ang,r_knee_ang,r_ankle_ang,r_elbow_ang,r_shoulder_ang,gender,age,r_hip_vel,...,r_hip_accel_2,r_knee_accel_2,r_ankle_accel_2,r_elbow_accel_2,r_shoulder_accel_2,r_hip_accel_3,r_knee_accel_3,r_ankle_accel_3,r_elbow_accel_3,r_shoulder_accel_3
0,10,1,167.768499,178.929176,142.051424,154.127553,23.245334,male,>40,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,10,2,165.381026,177.589431,137.767200,154.863261,23.917121,male,>40,-35.812098,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,10,3,164.644948,177.605573,133.497814,155.205706,24.322572,male,>40,-7.360778,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,10,4,163.897835,177.605651,132.629587,155.205706,24.802195,male,>40,-5.603347,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,10,5,163.620385,177.332893,132.012633,154.929898,25.170857,male,>40,-1.664701,...,-537.181477,-301.442616,-963.950478,165.534488,151.152007,0.0,0.0,0.0,0.0,0.0


In [4]:
gender_age = []
for i in range(len(df)):
    if df.iloc[i, 7] == 'male' and df.iloc[i, 8] == '>40':
        gender_age.append(0)
    elif df.iloc[i, 7] == 'male' and df.iloc[i, 8] == '<40':
        gender_age.append(1)
    elif df.iloc[i, 7] == 'female' and df.iloc[i, 8] == '>40':
        gender_age.append(2)
    elif df.iloc[i, 7] == 'female' and df.iloc[i, 8] == '<40':
        gender_age.append(3)

df.insert(9, "gender_age", gender_age)

In [5]:
df.head()

,id,frame,r_hip_ang,r_knee_ang,r_ankle_ang,r_elbow_ang,r_shoulder_ang,gender,age,gender_age,...,r_hip_accel_2,r_knee_accel_2,r_ankle_accel_2,r_elbow_accel_2,r_shoulder_accel_2,r_hip_accel_3,r_knee_accel_3,r_ankle_accel_3,r_elbow_accel_3,r_shoulder_accel_3
0,10,1,167.768499,178.929176,142.051424,154.127553,23.245334,male,>40,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,10,2,165.381026,177.589431,137.767200,154.863261,23.917121,male,>40,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,10,3,164.644948,177.605573,133.497814,155.205706,24.322572,male,>40,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,10,4,163.897835,177.605651,132.629587,155.205706,24.802195,male,>40,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,10,5,163.620385,177.332893,132.012633,154.929898,25.170857,male,>40,0,...,-537.181477,-301.442616,-963.950478,165.534488,151.152007,0.0,0.0,0.0,0.0,0.0


In [6]:
df.shape

(118248, 50)

In [7]:
df.iloc[:, 9].unique()

array([0, 1, 3, 2], dtype=int64)

In [8]:
# Check gender_age column

for i in range(len(df)):
    if df.iloc[i, 7] == "male" and df.iloc[i, 8] == ">40":
        assert df.iloc[i, 9] == 0, "(male, >40) should be 0"
    elif df.iloc[i, 7] == "male" and df.iloc[i, 8] == "<40":
        assert df.iloc[i, 9] == 1, "(male, <40) should be 1"
    elif df.iloc[i, 7] == "female" and df.iloc[i, 8] == ">40":
        assert df.iloc[i, 9] == 2, "(female, >40) should be 2"
    elif df.iloc[i, 7] == "female" and df.iloc[i, 8] == "<40":
        assert df.iloc[i, 9] == 3, "(female, <40) should be 3"
    else:
        raise Exception("There's something wrong with gender_age column")

In [9]:
# get the min frame number
min_frame = df.groupby("id")["frame"].nunique().min()

# make sure each subject has the same frame number
df = df[df.frame <= min_frame]

In [10]:
df.shape

(102443, 50)

In [11]:
X1 = df.iloc[:, 10:19]
X2 = df.iloc[:, 2:7]
X = pd.concat([X1, X2], axis=1)
y = df["gender_age"]

In [12]:
X.columns

Index(['r_hip_vel', 'r_knee_vel', 'r_ankle_vel', 'r_elbow_vel',
       'r_shoulder_vel', 'r_hip_accel', 'r_knee_accel', 'r_ankle_accel',
       'r_elbow_accel', 'r_hip_ang', 'r_knee_ang', 'r_ankle_ang',
       'r_elbow_ang', 'r_shoulder_ang'],
      dtype='object')

In [13]:
X = X.to_numpy()
y = y.to_numpy()

In [14]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [15]:
kf = StratifiedKFold(n_splits=3)
kf.get_n_splits(X, y)

3

In [16]:
def time_score(model, X_train, X_test, y_train, y_test):
  start = time.time()
  model.fit(X_train, y_train)
  end = time.time()
  y_pred = model.predict(X_test)
  return { "time": end - start, "accuracy": accuracy_score(y_test, y_pred) }
 

In [17]:
from sklearn.naive_bayes import BernoulliNB

In [18]:
bnb_scores = []

for train_index, test_index in kf.split(X, y):
  bnb_scores.append(time_score(BernoulliNB(), X[train_index], X[test_index], y[train_index], y[test_index]))

In [19]:
bnb_scores

[{'time': 0.03499865531921387, 'accuracy': 0.4190582171723088},
 {'time': 0.04204154014587402, 'accuracy': 0.4154855335597985},
 {'time': 0.03400111198425293, 'accuracy': 0.40864497613260314}]

In [20]:
# Average time
sum(list(map(lambda ts: ts["time"], bnb_scores))) / 3

0.03701376914978027

In [21]:
sum(list(map(lambda ts: ts["accuracy"], bnb_scores))) / 3

0.4143962422882368

In [22]:
from sklearn.tree import DecisionTreeClassifier

In [23]:
dtc_scores = []

for train_index, test_index in kf.split(X, y):
  dtc_scores.append(time_score(DecisionTreeClassifier(max_depth=200), X[train_index], X[test_index], y[train_index], y[test_index]))

In [24]:
dtc_scores

[{'time': 4.90472936630249, 'accuracy': 0.543370036312522},
 {'time': 5.228580713272095, 'accuracy': 0.6277087970012885},
 {'time': 5.6774163246154785, 'accuracy': 0.5089466131724603}]

In [25]:
sum(list(map(lambda ts: ts["time"], dtc_scores))) / 3

5.2702421347300215

In [26]:
sum(list(map(lambda ts: ts["accuracy"], dtc_scores))) / 3

0.5600084821620902

In [27]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
rfc_scores = []

for train_index, test_index in kf.split(X, y):
  rfc_scores.append(time_score(RandomForestClassifier(max_depth=10, n_estimators=100), X[train_index], X[test_index], y[train_index], y[test_index]))

In [29]:
rfc_scores

[{'time': 52.534324407577515, 'accuracy': 0.6016457772051071},
 {'time': 52.070318937301636, 'accuracy': 0.6371090547030572},
 {'time': 52.73174571990967, 'accuracy': 0.5420095469587373}]

In [30]:
sum(list(map(lambda ts: ts["time"], rfc_scores))) / 3

52.445463021596275

In [31]:
sum(list(map(lambda ts: ts["accuracy"], rfc_scores))) / 3

0.5935881262889672

In [32]:
from sklearn.ensemble import GradientBoostingClassifier

In [33]:
gbc_scores = []

for train_index, test_index in kf.split(X, y):
  gbc_scores.append(time_score(GradientBoostingClassifier(n_estimators=200), X[train_index], X[test_index], y[train_index], y[test_index]))

In [34]:
sum(list(map(lambda ts: ts["time"], gbc_scores))) / 3

907.0789831479391

In [35]:
sum(list(map(lambda ts: ts["accuracy"], gbc_scores))) / 3

0.5923290852623183

In [36]:
gbc_scores

[{'time': 917.0525693893433, 'accuracy': 0.5726836125102495},
 {'time': 889.7097039222717, 'accuracy': 0.6434930303385265},
 {'time': 914.4746761322021, 'accuracy': 0.560810612938179}]

In [37]:
from sklearn.neighbors import KNeighborsClassifier

In [38]:
knn_scores = []
for train_index, test_index in kf.split(X, y):
  knn_scores.append(time_score(KNeighborsClassifier(n_neighbors=10), X[train_index], X[test_index], y[train_index], y[test_index]))

In [39]:
sum(list(map(lambda ts: ts["time"], knn_scores))) / 3

0.4513734181722005

In [40]:
sum(list(map(lambda ts: ts["accuracy"], knn_scores))) / 3

0.6183529301249987

In [41]:
knn_scores

[{'time': 0.5690004825592041, 'accuracy': 0.5896685018156261},
 {'time': 0.3949930667877197, 'accuracy': 0.7178165631955019},
 {'time': 0.39012670516967773, 'accuracy': 0.547573725363868}]

In [42]:
from xgboost import XGBClassifier

In [43]:
xgbc_scores = []

xgbc = XGBClassifier(n_estimators=200, max_depth=10, learning_rate=1, objective='binary:logistic')
for train_index, test_index in kf.split(X, y):
  start = time.time()
  xgbc.fit(X[train_index], y[train_index])
  end = time.time()
  y_pred = xgbc.predict(X[test_index])
  xgbc_scores.append({"time": end - start, "accuracy": accuracy_score(y[test_index], y_pred)})

In [44]:
sum(list(map(lambda ts: ts["time"], xgbc_scores))) / 3

11.075249274571737

In [45]:
sum(list(map(lambda ts: ts["accuracy"], xgbc_scores))) / 3

0.6870056480866303

In [46]:
xgbc_scores

[{'time': 11.353684425354004, 'accuracy': 0.6643434461754715},
 {'time': 11.551730871200562, 'accuracy': 0.7900609113271642},
 {'time': 10.320332527160645, 'accuracy': 0.6066125867572554}]

In [47]:
from sklearn.svm import SVC

In [48]:
svc_scores = []

for train_index, test_index in kf.split(X, y):
  svc_scores.append(time_score(SVC(kernel='rbf', gamma=0.5, C=0.1), X[train_index], X[test_index], y[train_index], y[test_index]))

In [49]:
sum(list(map(lambda ts: ts["time"], svc_scores))) / 3

1160.5573070844014

In [50]:
sum(list(map(lambda ts: ts["accuracy"], svc_scores))) / 3

0.41948204998886857

In [51]:
svc_scores

[{'time': 1125.8223354816437, 'accuracy': 0.41911678575612044},
 {'time': 1142.8053817749023, 'accuracy': 0.4202003045566358},
 {'time': 1213.0442039966583, 'accuracy': 0.41912905965384956}]

In [ ]:
svc_scores = []

for train_index, test_index in kf.split(X, y):
  svc_scores.append(time_score(SVC(kernel='rbf', gamma=0.5, C=0.1), X[train_index], X[test_index], y[train_index], y[test_index]))